In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from astropy.io import fits

### Calculating bias

In [2]:
#function to get the data from fits files
def get_data(file):
    foo = fits.open(file)
    return foo[0].data

In [43]:
#function to calculate bias 
#code credit: https://stackoverflow.com/questions/10377998/how-can-i-iterate-over-files-in-a-given-directory
def bias_calc(directory):
    averages = []
    for filename in os.listdir(directory):
        if filename.endswith(".fits"):
              averages.append(get_data(directory + '/' + filename).mean())    
    avg = np.array([averages])
    bias = avg.mean()
    return bias

In [49]:
#directories for the bias data
bias_directory_008 = 'data/darks/0.008'
bias_directory_009 = 'data/darks/0.009'

In [50]:
#biases for 0.008 s and 0.009 s exposures 
bias_008 = bias_calc(bias_directory_008)
bias_009 = bias_calc(bias_directory_009)

In [51]:
print(bias_008)
print(bias_009)

766.3802704883889
766.4963268640007
